In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import learning_curve

data = pd.read_csv("cleaned_input/train_merged.csv")

In [12]:
#data = pd.DataFrame([[1,2,3,5,6,7,8,9],[1,3,9,7,5,4,5,7]], columns=['a','b','c','d','e','f','g','is_churn'])

In [13]:
model_data = data.sample(frac=.1)

In [14]:
X_train = model_data.drop('is_churn', axis=1);
y_train = model_data['is_churn'];

In [21]:
def plot_learning_curve(est, X_train, y_train):
    train_size, train_score, CV_score = learning_curve(est, X_train, y_train,train_sizes=np.linspace(0.1,1, 5));
    plt.figure();
    plt.plot(train_size, np.mean(train_score, axis=1), 'o-', color='b', label='training score');
    plt.plot(train_size, np.mean(CV_score, axis=1), 'o-', color='r', label='CV score');
    plt.xlabel("Training examples");
    plt.ylabel("Score");
    plt.fill_between(train_size, np.mean(train_score, axis=1) - np.std(train_score, axis=1),
                     np.mean(train_score, axis=1) + np.std(train_score, axis=1), alpha=0.1,
                     color="b");
    plt.fill_between(train_size, np.mean(CV_score, axis=1) - np.std(CV_score, axis=1),
                     np.mean(CV_score, axis=1) + np.std(CV_score, axis=1), alpha=0.1,
                     color="r");
    plt.title("Learning Curve", fontsize=20);
    plt.legend();
    plt.show();

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [23]:
#### making list of parameters for grid search
param_logit = {'C':[0.01,0.1,1,10,100], 'penalty':['l1','l2']}
param_knn = {'n_neighbors':[1,3,6,10], 'weights':['uniform','distance']}
param_svc = {'C':[0.01,0.1,1,10,100], 'kernel':{'rbf','sigmoid','precomputed'}}
#param_ann = {hidden_layer_sizes}
param_rfc = {'n_estimator':[5,10,15,20], 'max_features':[10,100,500,1000]}


In [24]:
model_list = {'LogisticRegression()':param_logit, 'KNeighborsClassifier()':param_knn,
             'SVC()':param_svc, 'RandomForestClassifier':param_rfc};


In [25]:
for model,param in model_list.items():
    gscv = GridSearchCV(model, param);
    print("learning curve for\t{}".format(model))
    plot_learning_curve(gscv, X_train, y_train)

learning curve for	LogisticRegression()


TypeError: 'module' object is not callable